In [ ]:
import requests
import base64

CLIENT_ID = '7d2df6c8c1b9435882308038dff6d342'
CLIENT_SECRET = '3f647d4b8fa447e8b2be350438fdaa48'

client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:

import pandas as pd
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):

    sp = spotipy.Spotify(auth=access_token)

    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']


        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None


        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None


        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None


        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None) if track_info else None,
            'External URLs': track_info.get('external_urls', {}).get('spotify', None) if track_info else None,
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,

        }

        music_data.append(track_data)


    df = pd.DataFrame(music_data)

    return df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.9 MB/s eta 0:00:00


In [ ]:
playlist_id = '7bUJaaksI2okk2ccSq0u2G'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                  Track Name  \
0                Mari Antaga   
1                Life Of Ram   
2              Nee Prashnalu   
3            Nammaka Tappani   
4   Ee Kshnam Oke Oka Korika   
..                       ...   
95             Nammave Ammai   
96                Alupannadi   
97            Nenani Neevani   
98  Nindu Noorella Version 2   
99            Cheppave Prema   

                                              Artists  \
0   Mickey J. Meyer, Sreerama Chandra, Sirivennela...   
1                                       Pradeep Kumar   
2                              S. P. Balasubrahmanyam   
3                                    Sagar, Sumangaly   
4                                       K. S. Chithra   
..                                                ...   
95                  Harish Raghavendra, K. S. Chithra   
96                                      K. S. Chithra   
97                                      Shweta Pandit   
98                         Gopika Poornima, K

In [ ]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [ ]:
def calculate_weighted_popularity(release_date):

    release_date = datetime.strptime(release_date, '%Y-%m-%d')


    time_span = datetime.now() - release_date


    weight = 1 / (time_span.days + 1)
    return weight

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return


    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]


    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)


    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]


    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [ ]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [26]:
import random

input_song_name = random.choice(music_df['Track Name'].tolist())

print(f"Recommending songs similar to: {input_song_name}")

recommendations = hybrid_recommendations(input_song_name)

print(recommendations)

Recommending songs similar to: Amma Brahma Devudo
                  Track Name  \
1                Jalsa Jalsa   
2            Bangaram Thechi   
0              Nammave Ammai   
4  Veyyinokka Jillala Varaku   
3     Intikeldam Padavammo..   

                                             Artists  \
1                      Baba Sehgal, Rita Thyagarajan   
2                      Partha Sarathy, K. S. Chithra   
0                  Harish Raghavendra, K. S. Chithra   
4                             S. P. Balasubrahmanyam   
3  S. P. Balasubrahmanyam, K. S. Chithra, Swarnal...   

                                     Album Name Release Date  Popularity  
1                                         Jalsa   2008-02-28        56.0  
2                               Iddaru Mithrulu   1999-04-30        35.0  
0                                         Vaasu   2002-03-10        31.0  
4                                     Surya IPS   1991-07-19        24.0  
3  Golden Hits Of Sirivennela Seetharama Sast